In [1]:
import openai
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score
import json
import os
from annotation_utils import get_assistant_texts, get_tag_masks, remove_tags, replace_tags, get_tags, get_message_ids
import time
from tqdm import tqdm
import threading
import queue
from itertools import islice

openai.api_key_path = "/home/alex/.personal/openAIkey"  # read protected to my 174 account

In [2]:
# a log of experiment results
with open("data/apt_tag_scores.json", "r") as f:
    average_scores = json.loads(f.read())

In [3]:
prompt_template = \
"""{}

###

Take the above text and place the tag `[[APT]]` at the end of every sentence or bullet point before any punctuation. For example:

# Example 1 #
Berries are a great source of antioxidants, vitamins and minerals[[APT]. They can help reduce inflammation, improve heart health, and lower cholesterol levels[[APT]].

# Example 2 #
Make a file called `hello.py` with the following contents:
```
if True:  # This always passes[[APT]]
    print("Hello, world!")  # This prints "Hello, world!" to the console[[APT]].
```
The `if` statement is always true, so the code will always run (it doesn't serve any purpose)[[APT]].

Do not tag questions as APT. Do not correct typos or add punctuation. Respond only with the input text, exactly unmodified other than the tags."""
# Truth Aptness
"""{}

Take the above input text and place the tag `[[APT]]` at the end of every statement that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. These will typically come at the end of sentences and bullet points, but not if they are commands or questions. Here are some examples:

# Example 1 #
Berries are a great source of antioxidants, vitamins and minerals[[APT]. They can help reduce inflammation, improve heart health, lower cholesterol levels, protect against certain cancers, support digestive health, boost immunity, and provide essential fatty acids[[APT]].

# Example 2 #
Make a file called `hello.py` with the following contents:
```
if True:
    print("Hello, world!")
```
This uses the builtin `print` function to print the string `Hello, world!` to the console[[APT]]. The `if` statement is always true, so the code will always run[[APT]] (it doesn't serve any purpose[[APT]]).

# Example 3 #
Tying a tie can seem tricky at first, but with practice, it becomes easier[[APT]]. Here's a step-by-step guide[[APT]]:
1. Cross the wide end over the narrow end.
2. Wrap the wide end behind the narrow end.
3. Bring the wide end back to the front.
4. Pass the wide end through the loop around your neck.
5. Tighten the knot and adjust as needed.

Respond only with the input text, exactly unmodified other than the tags."""

# # Example 2 #
# Certainly! A well-chosen color palette is one of the most important elements in creating a visually appealing website that effectively communicates your brand identity while also instilling confidence to potential visitors[[APT]]. Start by selecting one strong primary color that will dominate the entire website look. It's generally recommended to keep this color limited to 2-3 different shades or hues[[APT]].
"""{}

Take the above input text and place the tag `[[APT]]` at the end of every statement that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. These will typically come at the end of sentences and bullet points, but not if they are commands or questions. Here are some examples:

# Example 1 #
Berries are a great source of antioxidants, vitamins and minerals[[APT]. They can help reduce inflammation, improve heart health, lower cholesterol levels, protect against certain cancers, support digestive health, boost immunity, and provide essential fatty acids[[APT]].

# Example 2 #
Certainly! A well-chosen color palette is one of the most important elements in creating a visually appealing website that effectively communicates your brand identity while also instilling confidence to potential visitors[[APT]]. Start by selecting one strong primary color that will dominate the entire website look. It's generally recommended to keep this color limited to 2-3 different shades or hues[[APT]].

# Example 3 #
Tying a tie can seem tricky at first, but with practice, it becomes easier[[APT]]. Here's a step-by-step guide[[APT]]:
1. Cross the wide end over the narrow end.
2. Wrap the wide end behind the narrow end.
3. Bring the wide end back to the front.
4. Pass the wide end through the loop around your neck.
5. Tighten the knot and adjust as needed.

Respond only with the input text, exactly unmodified other than the tags."""


"""{}

Take the above input text and place the tag `[[APT]]` at the end of every statement that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. These will typically come at the end of sentences and bullet points, but not if they are commands or questions. Here are some examples:

# Example 1 #
Berries are a great source of antioxidants, vitamins and minerals[[APT]. They can help reduce inflammation, improve heart health, lower cholesterol levels, protect against certain cancers, support digestive health, boost immunity, and provide essential fatty acids[[APT]].

# Example 2 #
Certainly! A well-chosen color palette is one of the most important elements in creating a visually appealing website that effectively communicates your brand identity while also instilling confidence to potential visitors[[APT]]. Start by selecting one strong primary color that will dominate the entire website look. It's generally recommended to keep this color limited to 2-3 different shades or hues[[APT]].

# Example 3 #
The most common ways to treat a cold include:
- Resting and drinking plenty of fluids[[APT]]
- Taking over-the-counter medications such as acetaminophen or ibuprofen[[APT]]
- Using a humidifier to help relieve congestion[[APT]]

Respond only with the input text, exactly unmodified other than the tags."""

"""{}

Take the above input text and place the tag `[[APT]]` at the end of every statement/fragment that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. Here are some examples:

# Example 1 #
Berries are a great source of antioxidants[[APT]], vitamins and minerals[[APT]. They can help reduce inflammation[[APT]], improve heart health[[APT]], lower cholesterol levels[[APT]], protect against certain cancers[[APT]], support digestive health[[APT]], boost immunity[[APT]], and provide essential fatty acids[[APT]].

# Example 2 #
The bitter lesson focuses on a company that hired an executive with false credentials[[APT]], who then proceeded to bring the company into ruin through deception and manipulation[[APT]]. The author argues that companies should not hire executives from outside their own industry[[APT]]; they will always be at a disadvantage[[APT]] due to unfamiliarity with corporate politics and practices[[APT]], which leaves them vulnerable to being manipulated or deceived[[APT]]. In contrast, hiring internal candidates shows true dedication to the long-term success of the company[[APT]]. Additionally, the author suggests that it is valuable for leaders within corporations to learn about different industries[[APT]] so as to better understand how various business decisions impact overall performance[[APT]]. The book ends with advice for improving ethical decision making skills[[APT]] in order to avoid similar missteps in the future[[APT]].

Respond only with the input text, exactly unmodified other than the tags. Apply tags generously, even to sentence fragments, as shown in the examples."""


'{}\n\nTake the above input text and place the tag `[[APT]]` at the end of every statement/fragment that is truth-apt. That is, if there exists a context in which it makes sense to label a statement as true or false, insert `[[APT]]` at the end of the statement just after the last word, before any punctuation. Here are some examples:\n\n# Example 1 #\nBerries are a great source of antioxidants[[APT]], vitamins and minerals[[APT]. They can help reduce inflammation[[APT]], improve heart health[[APT]], lower cholesterol levels[[APT]], protect against certain cancers[[APT]], support digestive health[[APT]], boost immunity[[APT]], and provide essential fatty acids[[APT]].\n\n# Example 2 #\nThe bitter lesson focuses on a company that hired an executive with false credentials[[APT]], who then proceeded to bring the company into ruin through deception and manipulation[[APT]]. The author argues that companies should not hire executives from outside their own industry[[APT]]; they will always be

In [4]:
# # load the validation dataset

# ann_path = "data/validation_annotations.txt"
# with open(ann_path) as f:
#     annotated = f.read()

# to_replace = ("LE", "LH", "NORM", "APT", "IMP")
# message_ids = get_message_ids(annotated)
# annotated_assistant_texts = replace_tags(get_assistant_texts(annotated), to_replace=to_replace)
# assistant_texts = remove_tags(annotated_assistant_texts)
# val_tag_masks = [get_tag_masks(annotated_assistant_text).get("APT", [0] * len(assistant_text)) for annotated_assistant_text, assistant_text in zip(annotated_assistant_texts, assistant_texts)]
# list(zip(annotated_assistant_texts, assistant_texts))

# load the inference dataset
# texts = []
# for path in os.listdir("data/oasst/transcripts/original"):
#     with open(os.path.join("data/oasst/transcripts/original", path)) as f:
#         texts.append(f.read())

# text = "\n\n\n".join(texts[684:])
# assistant_texts = get_assistant_texts(text)
# message_ids = get_message_ids(text)
# assert len(assistant_texts) == len(message_ids)
# print(len(message_ids))

# weak_lm_responses = pd.read_csv("data/transcripts_vicgalle_gpt2-open-instruct-v1(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).csv")
# weak_lm_responses = pd.read_csv("data/transcripts_keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).csv").iloc[:50]
# message_ids = weak_lm_responses["parent_id"].apply(lambda x: "P=" + x).values.tolist()
# assistant_texts = weak_lm_responses["response"].values.tolist()
# parent_texts = weak_lm_responses["parent_text"].values.tolist()
# print(len(assistant_texts))

# load inference dataset (training dataset for deberta)
path = "data/oasst/2023-04-12_oasst_all.messages.jsonl"
inf_df = pd.read_json(path, lines=True)
inf_df = inf_df[(inf_df["role"] == "assistant") & (inf_df["lang"] == "en") & (inf_df["tree_state"] == "ready_for_export") & ~inf_df["deleted"]]
message_ids = inf_df["message_id"].values.tolist()
assistant_texts = inf_df["text"].values.tolist()

message_ids[:4]

['c8e83833-ecbc-44fe-b6db-735228c25a1c',
 '343ee2d4-87ae-41fd-a768-bdd65959dc4a',
 '636dd191-50df-4894-ba9a-cd7f00767258',
 '73d6f715-3787-409c-81e4-fde0e5ef60cd']

In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")

model_name = "gpt-3.5-turbo"
temperature = 0
tagging_eagerness_bias = 0
# add this much to the "[[" logit to make it eager to tag
logit_bias = {tokenizer.encode("[[")[0]: tagging_eagerness_bias}
stop_seq = prompt_template[4:55]  # "Take the above input text and place the tag [[APT]]"

API_costs = {
    "gpt-3.5-turbo": {"prompt_tokens": 0.0015 / 1000, "completion_tokens": 0.002 / 1000},
    "gpt-4": {"prompt_tokens": 0.03 / 1000, "completion_tokens": 0.06 / 1000},
}

/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
# use gpt2 tokenizer to get an estimate for the number of tokens the model needs to complete (GPT3 uses the same tokenizer)
# consider upweighting the "[[" logits

# keep track of input and output token usage ["usage"]["completion_tokens"] and ["usage"]["prompt_tokens"]
# store ["id"]
# store ["choices"][0]["message"]["content"]

def tag(i, message_id, example, results, **kwargs):
    try:
        example_tokens = len(tokenizer.encode(example))
        input = prompt_template.format(example)
        for i in range(5):
            try:
                if i > 0:
                    print("Retrying request")
                
                completion = openai.ChatCompletion.create(
                    model=model_name,
                    messages=[
                        {"role": "system", "content": "You are a helpful assistant."},
                        {"role": "user", "content": input},
                    ],
                    temperature=temperature,
                    max_tokens=int(example_tokens * 1.5) + 5 + len(stop_seq),  # should just be a copy of example with a few tokens added
                    logit_bias=logit_bias,
                    stop=stop_seq,
                )
                break
            except Exception as e:
                print("Error completing request:", e)
                time.sleep(2)
        
        usage = completion["usage"]
        prompt_tokens, completion_tokens = usage["prompt_tokens"], usage["completion_tokens"]
        cost = API_costs[model_name]["prompt_tokens"] * prompt_tokens + API_costs[model_name]["completion_tokens"] * completion_tokens
        
        # check that there's only one choice, and the ["choices"][0]["message"]["role"] is "assistant"
        if len(completion["choices"]) != 1:
            print("SKIPPING: multiple choices")
            return
        if completion["choices"][0]["message"]["role"] != "assistant":
            print("SKIPPING: role is not assistant")
            return
        # check that finish reason is not for a content filter, not for length, not for function_call and that it is "stop"
        if completion["choices"][0]["finish_reason"] != "stop":
            print(f"SKIPPING: finish reason is {completion['choices'][0]['finish_reason']}, not stop")
            print("RESPONSE:", completion["choices"][0]["message"]["content"])
            return

        response = completion["choices"][0]["message"]["content"]
        if response.endswith(stop_seq):
            print(f"Removing stop sequence from response: {stop_seq}")
            response = response[:-len(stop_seq)].rstrip()

        response = response.strip()

        # check that the response is an exact match to the prompt
        clean_response = remove_tags(response)
        response_tags = get_tags(response).get("APT", [])
        response_tag_mask = get_tag_masks(response).get("APT", [0] * len(clean_response))
        
        if clean_response.strip() != example.strip():
            print(f"SKIPPING: response does not match prompt:\n")
            print("EXAMPLE:")
            print(example)
            print("RESPONSE:")
            print(clean_response)
            return
            
        
        result = {
            "message_id": message_id,
            "completion_id": completion["id"],
            # "user_prompt": user_prompt,
            "input": input,
            "example": example,
            "response": response,
            "prompt_tokens": prompt_tokens,
            "completion_tokens": completion_tokens,
            "dollars": cost,
            "pred_tags": response_tags,
            "pred_tag_mask": response_tag_mask,
            **kwargs,
        }
        results.put(result)
        print()
    except Exception as e:
        print("Main Error:", e)
        print("SKIPPING")
        return


results = queue.Queue()
total_cost = 0
n_threads = min(10, len(message_ids))
n_iter = (len(message_ids) // n_threads) * n_threads
iterator = islice(enumerate(zip(message_ids, assistant_texts)), n_iter)

In [7]:
while True:
    threads = []
    for _ in range(n_threads):
        i, (message_id, example, *rest) = next(iterator)
        t = threading.Thread(target=tag, args=(i, message_id, example, results, *rest))
        threads.append(t)
        t.start()

    # make sure they don't take more than 10 seconds
    for t in threads:
        t.join(timeout=60)
    
    for t in threads:
        if t.is_alive():
            print("THREAD TIMED OUT")
            try:
                t._stop()
            except AssertionError:
                print("Thread could not be terminated")


    # compute total cost
    print(f"###### {i} ######")
    total_cost = sum([r["dollars"] for r in results.queue])
    print(f"Total cost: ${total_cost:.4f}")
    answer_rate = len(results.queue) / (i + 1)
    print(f"Answer rate: {answer_rate:.4f}")
    projected_cost = total_cost / answer_rate * len(message_ids) / (i + 1)
    print(f"Projected cost: {projected_cost:.4f}")

    step = 4 * n_threads
    if (i + 1) % step == 0:
        df = pd.DataFrame(list(results.queue))
        df.to_json(f"data/apt_tagger_training_data/{model_name}_{i + 1}.json", orient="records")
        # remove previos checkpoint
        if os.path.exists(f"data/apt_tagger_training_data/{model_name}_{i + 1 - step}.json"):
            os.remove(f"data/apt_tagger_training_data/{model_name}_{i + 1 - step}.json")
    if i == n_iter - 1:
        break
    
df = pd.DataFrame(list(results.queue))




SKIPPING: response does not match prompt:

EXAMPLE:
Here are some potential regulatory options to address monopsony power over employees:

- Antitrust enforcement agencies like the FTC or DOJ could investigate and take action against dominant employers that are abusing their market power to suppress wages or worsen working conditions. They could force divestitures or other structural remedies.
- Labor standards agencies could set industry-specific minimum wage floors or other regulations around compensation and working conditions. This would reduce the ability of dominant employers to push wages/conditions below certain thresholds.
- Sector-specific regulators could impose requirements around how companies hire, compensate, and treat workers. For example, banking regulators could prohibit abusive practices by dominant banks with regard to their employees.
- Legislatures could pass laws
RESPONSE:
Here are some potential regulatory options to address monopsony power over employees:

-

Token indices sequence length is longer than the specified maximum sequence length for this model (1651 > 1024). Running this sequence through the model will result in indexing errors


SKIPPING: response does not match prompt:

EXAMPLE:
To achieve Turing completeness, a system must have the ability to perform arbitrary computations, meaning it can execute any algorithm that can be described in a finite number of steps. Here are some key features that I am missing to be considered Turing complete:

1. Universal computation: A Turing complete system must be able to execute any computable function, whereas I have been trained on a specific task and have limitations in the types of computations I can perform.

2. Programmability: A Turing complete system must have the ability to execute user-defined algorithms, whereas I can only generate text based on patterns I've seen in my training data.

3. Memory: A Turing complete system must have the ability to store and retrieve data from memory, whereas I can only access information that has been provided to me in each individual request.

4. Conditionals and loops: A Turing complete system must have the ability to perform cond

KeyboardInterrupt: 




SKIPPING: response does not match prompt:

EXAMPLE:
I'm sorry I don't have access to that information and there may be multiple recordings with different narrators or even abridged versions with different chapter lengths. 

You may be able to find this information by looking at a few audiobook resellers as well as free projects like librivox. There are sometimes free readings of popular and out-of-copyright books on youtube or podcasts so you can also search there.
RESPONSE:
I'm sorry I don't have access to that information and there may be multiple recordings with different narrators or even abridged versions with different chapter lengths.

You may be able to find this information by looking at a few audiobook resellers as well as free projects like librivox. There are sometimes free readings of popular and out-of-copyright books on youtube or podcasts so you can also search there.








In [ ]:
new_df = pd.DataFrame(list(results.queue))

In [ ]:
sorted_val_tag_masks = [val_tag_masks[message_ids.index(message_id)] for message_id in df["message_id"].values]
cat_pred_masks = np.concatenate(df["pred_tag_mask"].values, axis=0)
cat_gt_masks = np.concatenate(sorted_val_tag_masks, axis=0)
weighted_prec = precision_score(cat_gt_masks, cat_pred_masks)
weighted_rec = recall_score(cat_gt_masks, cat_pred_masks)
weighted_acc = accuracy_score(cat_gt_masks, cat_pred_masks)
weighted_f1 = f1_score(cat_gt_masks, cat_pred_masks)
print("weighted precision:", weighted_prec)
print("weighted recall:", weighted_rec)
print("weighted accuracy:", weighted_acc)
print("weighted f1:", weighted_f1)

weighted precision: 0.9710144927536232
weighted recall: 0.9305555555555556
weighted accuracy: 0.999161576236675
weighted f1: 0.9503546099290779


In [128]:
df = pd.DataFrame(list(results.queue))
df.to_json(f"data/oasst_apt_tags_{model_name}_{i + 1}.json")

1.0

In [129]:
print("estimated total cost for 50k:", total_cost * (50000 / len(assistant_texts)) * len(assistant_texts) / len(new_df))

estimated total cost for 50k: 49.233333333333334


In [130]:
score_object = {
    "prompt_template": prompt_template,
    "model_name": model_name,
    "temperature": temperature,
    "tagging_eagerness_bias": tagging_eagerness_bias,
    "weighted_accuracy": weighted_acc,
    "weighted_precision": weighted_prec,
    "weighted_recall": weighted_rec,
    "weighted_f1": weighted_f1,
    "skip_rate": 1 - (len(df) / len(assistant_texts)),
    "total_cost": total_cost,
}
average_scores.append(score_object)
# make a copy of the file we're about to write if it exists
if os.path.exists("data/apt_tag_scores.json"):
    os.rename("data/apt_tag_scores.json", "data/apt_tag_scores.json.bak")
with open("data/apt_tag_scores.json", "w") as f:
    f.write(json.dumps(average_scores, indent=2))
score_object

{'prompt_template': '{}\n\n###\n\nTake the above text and place the tag `[[APT]]` at the end of every sentence or bullet point before any punctuation. For example:\n\n# Example 1 #\nBerries are a great source of antioxidants, vitamins and minerals[[APT]. They can help reduce inflammation, improve heart health, and lower cholesterol levels[[APT]].\n\n# Example 2 #\nMake a file called `hello.py` with the following contents:\n```\nif True:  # This always passes[[APT]]\n    print("Hello, world!")  # This prints "Hello, world!" to the console[[APT]].\n```\nThe `if` statement is always true, so the code will always run (it doesn\'t serve any purpose)[[APT]].\n\nDo not tag questions as APT. Do not correct typos or add punctuation. Respond only with the input text, exactly unmodified other than the tags.',
 'model_name': 'gpt-3.5-turbo',
 'temperature': 0,
 'tagging_eagerness_bias': 0,
 'weighted_accuracy': 0.999161576236675,
 'weighted_precision': 0.9710144927536232,
 'weighted_recall': 0.930

In [131]:
resp

"A Value-Added Tax (VAT) is a tax that is levied on goods and services at the time of sale, rather than when they are purchased[[APT]]. The main advantage of a VAT is that it eliminates the need for an income tax, as all revenue generated from the tax can be used to finance public spending[[APT]].\n\nThe effects of abolishing the US income tax and replacing it with a VAT could vary depending on several factors, such as the specific design of the new tax system, its level of taxation, and its impact on economic growth and employment[[APT]]. Here's some possible scenarios:\n\nScenario 1: No change in overall taxes\nUnder this scenario, there would be no major changes to the existing tax system[[APT]]. Income taxes still exist, but their rates and brackets would remain unchanged[[APT]]. This could lead to a slight increase in total tax collections, as more people pay into the system through higher wages or profits[[APT]]. However, this would likely not have a significant effect on the ove

In [132]:
gold

"A Value-Added Tax (VAT) is a tax that is levied on goods and services at the time of sale, rather than when they are purchased[[APT]]. The main advantage of a VAT is that it eliminates the need for an income tax, as all revenue generated from the tax can be used to finance public spending[[APT]].\n\nThe effects of abolishing the US income tax and replacing it with a VAT could vary depending on several factors, such as the specific design of the new tax system, its level of taxation, and its impact on economic growth and employment[[APT]]. Here's some possible scenarios:\n\nScenario 1: No change in overall taxes\nUnder this scenario, there would be no major changes to the existing tax system[[APT]]. Income taxes still exist, but their rates and brackets would remain unchanged[[APT]]. This could lead to a slight increase in total tax collections, as more people pay into the system through higher wages or profits[[APT]]. However, this would likely not have a significant effect on the ove

In [133]:
for i in range(len(df)):
    print(i)
    resp = df.response.iloc[i]
    gold = annotated_assistant_texts[message_ids.index(df.message_id.iloc[i])]
    if gold != resp:
        print("RESP:", resp)
        print("GOLD:", gold)
        # break
        print("\n")

0
1
2
RESP: Buying a house has several tax implications compared to renting. First, you will likely have to pay capital gains taxes on any profit from selling the property[[APT]]. Second, you may be subject to higher income taxes if your income is high enough[[APT]]. Third, you may also be liable for mortgage insurance premiums and other fees associated with owning a home[[APT]]. Fourth, depending on where you live, you may need to pay local property taxes or school taxes[[APT]]. Finally, you may be able to deduct some expenses related to owning a home such as mortgage interest and property taxes[[APT]].
GOLD: Buying a house has several tax implications compared to renting[[APT]]. First, you will likely have to pay capital gains taxes on any profit from selling the property[[APT]]. Second, you may be subject to higher income taxes if your income is high enough[[APT]]. Third, you may also be liable for mortgage insurance premiums and other fees associated with owning a home[[APT]]. Four

# Make the validation set

In [1]:
from AptTagger import AptTagger
from TruthLabeler import TruthLabeler
from annotation_utils import make_transcript
import pandas as pd

path1 = "data/oasst/generated_responses/50_transcripts_keyfan_bloomz-rlhf(max_length=564;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"
path2 = "data/oasst/generated_responses/50_transcripts_OpenAssistant_oasst-sft-4-pythia-12b-epoch-3.5(max_length=533;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"

df1 = pd.read_json(path1).iloc[:50]
df2 = pd.read_json(path2).iloc[-1:]
brennan_df = pd.concat([df1, df2])

apt_tagger = AptTagger(device="cuda:7")

def add_ann_cols(df):
    df["tag_mask"] = apt_tagger.tag(df["assistant_text"])
    df["annotated_assistant_text"] = df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
    df["annotated_transcript"] = df.apply(make_transcript, axis=1, args=(True,))  # True for annotated
    df["templated_annotated_transcript"] = df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])

add_ann_cols(brennan_df)
brennan_df.head()


/mnt/ssd-2/spar/alexm/miniconda3/envs/dlkb/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,prompt,assistant_text,parent_id,parent_text,prev_messages,role,synthetic,model,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript
0,A chat between a curious human and an artifici...,"First, try and prioritize your tasks based on ...",f51d172d-4923-4eaf-8883-f0118c898182,can you be more specific? i think a bit of gui...,[I'm completely overwhelmed with all the work ...,assistant,True,keyfan/bloomz-rlhf(max_length=693;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","First, try and prioritize your tasks based on ...",PROMPTER: I'm completely overwhelmed with all ...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
1,A chat between a curious human and an artifici...,"The biggest problem with the Burj Khalifa, my ...",113af868-3868-4276-86dc-dc465d7875b9,What is the biggest problem with the Burj Khal...,[What is the biggest problem with the Burj Kha...,assistant,True,keyfan/bloomz-rlhf(max_length=566;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","The biggest problem with the Burj Khalifa, my ...",PROMPTER: What is the biggest problem with the...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
2,A chat between a curious human and an artifici...,There are many things you can do in Rome that ...,336fb154-f62d-4f01-8fe3-7002b3b49389,I've never been to Rome. What must I see?,[I've never been to Rome. What must I see?],assistant,True,keyfan/bloomz-rlhf(max_length=557;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There are many things you can do in Rome that ...,PROMPTER: I've never been to Rome. What must I...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
3,A chat between a curious human and an artifici...,Certainly! Here is what that might look like u...,a2703c6e-0a59-4c54-97b3-50b538502d14,Can you do it again but in Java?,"[Write a simple Hello World program in python,...",assistant,True,keyfan/bloomz-rlhf(max_length=596;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! Here is what that might look like u...,PROMPTER: Write a simple Hello World program i...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
4,A chat between a curious human and an artifici...,Here are some potential ways to encourage more...,9a001945-07a7-4030-8bfd-1994cc42554b,How can I convince someone to be proactive who...,[How can I convince someone to be proactive wh...,assistant,True,keyfan/bloomz-rlhf(max_length=597;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Here are some potential ways to encourage more...,PROMPTER: How can I convince someone to be pro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


In [2]:
from hashlib import md5
brennan_df["message_id"] = brennan_df["assistant_text"].apply(lambda x: md5(x.encode()).hexdigest())
brennan_df.head()

,prompt,assistant_text,parent_id,parent_text,prev_messages,role,synthetic,model,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript,message_id
0,A chat between a curious human and an artifici...,"First, try and prioritize your tasks based on ...",f51d172d-4923-4eaf-8883-f0118c898182,can you be more specific? i think a bit of gui...,[I'm completely overwhelmed with all the work ...,assistant,True,keyfan/bloomz-rlhf(max_length=693;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","First, try and prioritize your tasks based on ...",PROMPTER: I'm completely overwhelmed with all ...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,36ccb4cb2be532e9170710eb8819a393
1,A chat between a curious human and an artifici...,"The biggest problem with the Burj Khalifa, my ...",113af868-3868-4276-86dc-dc465d7875b9,What is the biggest problem with the Burj Khal...,[What is the biggest problem with the Burj Kha...,assistant,True,keyfan/bloomz-rlhf(max_length=566;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","The biggest problem with the Burj Khalifa, my ...",PROMPTER: What is the biggest problem with the...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,df951205cc75dbbb35bcbd1314d18911
2,A chat between a curious human and an artifici...,There are many things you can do in Rome that ...,336fb154-f62d-4f01-8fe3-7002b3b49389,I've never been to Rome. What must I see?,[I've never been to Rome. What must I see?],assistant,True,keyfan/bloomz-rlhf(max_length=557;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",There are many things you can do in Rome that ...,PROMPTER: I've never been to Rome. What must I...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,39dcc0f1ed81f493eb2d21e1e6a46137
3,A chat between a curious human and an artifici...,Certainly! Here is what that might look like u...,a2703c6e-0a59-4c54-97b3-50b538502d14,Can you do it again but in Java?,"[Write a simple Hello World program in python,...",assistant,True,keyfan/bloomz-rlhf(max_length=596;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! Here is what that might look like u...,PROMPTER: Write a simple Hello World program i...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,23d7ff066f9971afb04577aca78a35ba
4,A chat between a curious human and an artifici...,Here are some potential ways to encourage more...,9a001945-07a7-4030-8bfd-1994cc42554b,How can I convince someone to be proactive who...,[How can I convince someone to be proactive wh...,assistant,True,keyfan/bloomz-rlhf(max_length=597;temperature=...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Here are some potential ways to encourage more...,PROMPTER: How can I convince someone to be pro...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...,b2306c0c6977ed4b360ed1291b7af51c


In [3]:
print(brennan_df["templated_annotated_transcript"].iloc[0])

### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
PROMPTER: I'm completely overwhelmed with all the work I need to do. So many people expect stuff form me and I just can't handle it all. The more I work, the less I get done and I have so many unanswered email and things I promised to do, I'm scared to even go look at my inbox. What can I do?

ASSISTANT: The best thing you could do in this situation is to take a day off and relax somewhere, you could read a book or lay in bed watching YouTube or you could talk to someone. But do make sure that you're not missing anything important that has been planned on that day.

PROMPTER: can you be more specific? i think a bit of guidance could help me..

ASSISTANT: First, try and prioritize your tasks based on urgency and importance, then break them down into smaller manageable chunks[[1]]. This will allow you to tackle one task at a time instead of feeling overwhelmed by everything at once[[2]]. Also try and schedule some breaks while working, as lo

In [4]:
import pandas as pd
old_combined_df = pd.read_json("data/label-validation/old_tagging_style/transcripts_combined.json")
old_combined_df.drop(columns=["parent_text", "role", "rank", "user_text"], inplace=True)

add_ann_cols(old_combined_df)
old_combined_df.head()

,prompt,response,parent_id,synthetic,model,assistant_text,message_id,prev_messages,tag_mask,annotated_assistant_text,annotated_transcript,templated_annotated_transcript
0,A chat between a curious human and an artifici...,"As an artificial intelligence language model, ...",6faa03e5-5f0c-483c-9d22-a15505a2b821,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",87cdf227-c048-4ccb-a790-0f44dc5ebb6b,[Where was your last vaccation?],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",PROMPTER: Where was your last vaccation?\n\nAS...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
1,A chat between a curious human and an artifici...,Joseph Scaliger (1557-1636) was an Italian sch...,2f2e3cfd-857f-41b5-9824-0b62f7dd374c,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,[Describe Joseph Scaliger and his contribution...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,PROMPTER: Describe Joseph Scaliger and his con...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
2,A chat between a curious human and an artifici...,Certainly! A well-chosen color palette is one ...,89b13be2-c82f-493a-9258-1d690440e451,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,69f27058-6253-42bc-892c-5f8a95870ec1,"[I am designing a website, can you give me a c...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"PROMPTER: I am designing a website, can you gi...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
3,A chat between a curious human and an artifici...,Lena was a happy-go-lucky suburban housewife w...,0cd84750-ccf6-4a8e-89bf-02e17a6d0eca,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,b864688c-62cb-420d-bae1-e81677b4fded,[Write me a story synopsis for a novel where t...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,PROMPTER: Write me a story synopsis for a nove...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
4,A chat between a curious human and an artifici...,I apologize if my previous response did not ac...,dfe226cd-1663-45e3-ab80-7b6065da46ac,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,I apologize if my previous response did not ac...,0412dc3f-ae56-4d83-b361-cb1fa7c248a6,[A ball was thrown on a planet with a density ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",I apologize if my previous response did not ac...,PROMPTER: A ball was thrown on a planet with a...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


# Combine val sets

In [9]:
# (60) first make 15 new examples for each dataset, model in {(oasst, vicuna), (alpaca, pythia), (alpaca, bloomz), (alpaca, vicuna)}
# (51) then re-apt-tag Brennan's split (only the first half + 1)
# (21) and the 21 in auto-labeling/data/label-validation/old_tagging_style/transcripts_combined.json

# make the 60 new examples
paths = [
    "data/oasst/generated_responses/6000_transcripts_alpaca_OpenAssistant_oasst-sft-4-pythia-12b-epoch-3.5(max_length=2048;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json",
    "data/oasst/generated_responses/6000_transcripts_alpaca_lmsys_vicuna-7b-v1.5(max_length=4096;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json",
    "data/oasst/generated_responses/6000_transcripts_alpaca_keyfan_bloomz-rlhf(max_length=2048;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json",
    "data/oasst/generated_responses/5999_transcripts_oasst_lmsys_vicuna-7b-v1.5(max_length=4096;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).json"
]
n_each = 15
dfs = [pd.read_json(path).iloc[:n_each] for path in paths]
cat_df = pd.concat(dfs)

add_ann_cols(cat_df)
print(cat_df["templated_annotated_transcript"].iloc[0])

### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###
PROMPTER: You are required to analyze the following financial statement for any discrepancies.

The following financial statement includes total revenue of $13,000, total expenses of $10,000, and net income of $3,000.

ASSISTANT: Based on this financial statement there appears to be a discrepancy between total revenue ($13,000) and total expenses ($10,000)[[1]]. It is unclear from the information provided whether or not all costs have been properly accounted for in order to calculate the Net Income of $3000[[2]]. Additionally, it would be helpful to know how these numbers were calculated (e.g., by using gross profit margin or operating margin[[3]]?), as well as an explanation of why there was a difference between Total Revenue and Total Expenses[[4]]. Finally, if possible, provide an analysis of this comparison in terms of areas where further review could help improve financial performance[[5]].
###

First, decide whether each statement ca

In [10]:
old_combined_df["subset_name"] = "old_combined"
brennan_df["subset_name"] = "brennan"
cat_df["subset_name"] = "new_combined"
all_df = pd.concat([old_combined_df, brennan_df, cat_df])
len(all_df)

132

In [11]:
out_str = ""
for row in all_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[SUBSET_NAME]] {row['subset_name']}\n"
    out_str += f"[[INPUT_TEXT]] {row['templated_annotated_transcript']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/label-validation/BLANK_val_labels_all.txt"
with open(out_file, "w") as f:
    f.write(out_str)

all_df.to_json("data/label-validation/all_transcripts.json", orient="records")

# clean up dataframes

In [33]:
model_name = "keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95)"
data_path = f"data/transcripts_{model_name}.csv"
df = pd.read_csv(data_path)

In [34]:
print(f"Total number of transcripts: {len(df)}")
df = df.drop_duplicates(subset=['prompt'])
df = df.reset_index(drop=True)
len(df)

Total number of transcripts: 843


831

In [35]:
df["response"].iloc[0]

"Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being 

In [36]:
import uuid
def get_asst_text(text):
    if "smartplat" in data_path:
        bits = text.split("ASSISTANT:")
        bits = bits[1].split("USER:")
        bits = bits[0].split("</s>")  # TODO: figure out why the model is outputting this
        return bits[0].strip()
    elif "keyfan_bloomz" in data_path:
        return text
    else:
        raise Exception("Unknown dataset")

df["assistant_text"] = df["response"].apply(get_asst_text)
df["user_text"] = df["parent_text"]
df["message_id"] = [str(uuid.uuid4()) for _ in range(len(df))]
df["assistant_text"].iloc[0], df["user_text"].iloc[0]

("Sure! DHCP (Dynamic Host Configuration Protocol) is a protocol used by network devices such as routers to assign IP addresses to client computers or other networked devices.\n\nDHCP assigns each device on a local area network (LAN) with a unique public address called an Internet Protocol (IP) address. This makes it possible for all networked devices on the LAN to communicate with one another using standard internet protocols like TCP/IP.\n\nBy default, most operating systems come preconfigured with static IP configurations meaning that the assigned IP address remains constant until manually changed through configuration changes in the system settings of your router or modem or even via software installations from certain sources.  When this happens, it's common practice for people still not familiar enough with DHCP to assume that they have manually set up their PCs with dynamic DNS so they are always downloading web applications directly off the internet and thus automatically being

In [37]:
df.to_json(f"data/transcripts_{model_name}.json", orient="records")

In [38]:
from TruthLabeler import TruthLabeler
from AptTagger import AptTagger
apt_tagger = AptTagger()

In [42]:
df.iloc[16].parent_id

'dfe226cd-1663-45e3-ab80-7b6065da46ac'

In [39]:
raw_df = df.iloc[11:21]
raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
raw_df["annotated_transcript"] = raw_df.apply(lambda x: f"USER: {x['user_text']}\n\nASSISTANT: {x['annotated_assistant_text']}", axis=1)
raw_df["input_text"] = raw_df["annotated_transcript"].apply(lambda x: TruthLabeler.make_input(x)[0])
raw_df.head()

/tmp/ipykernel_195852/416794750.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["tag_mask"] = apt_tagger.tag(raw_df["assistant_text"])
/tmp/ipykernel_195852/416794750.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_df["annotated_assistant_text"] = raw_df.apply(lambda x: apt_tagger.annotate(x["assistant_text"], x["tag_mask"]), axis=1)
/tmp/ipykernel_195852/416794750.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

,prompt,response,parent_id,parent_text,role,synthetic,model,assistant_text,user_text,message_id,tag_mask,annotated_assistant_text,annotated_transcript,input_text
11,A chat between a curious human and an artifici...,"As an artificial intelligence language model, ...",6faa03e5-5f0c-483c-9d22-a15505a2b821,Where was your last vaccation?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,"As an artificial intelligence language model, ...",Where was your last vaccation?,87cdf227-c048-4ccb-a790-0f44dc5ebb6b,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","As an artificial intelligence language model, ...",USER: Where was your last vaccation?\n\nASSIST...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
12,A chat between a curious human and an artifici...,Joseph Scaliger (1557-1636) was an Italian sch...,2f2e3cfd-857f-41b5-9824-0b62f7dd374c,Describe Joseph Scaliger and his contributions...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Joseph Scaliger (1557-1636) was an Italian sch...,Describe Joseph Scaliger and his contributions...,fd9fa103-6988-4d4c-b6de-2e7a0ac8b5a9,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Joseph Scaliger (1557-1636) was an Italian sch...,USER: Describe Joseph Scaliger and his contrib...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
13,A chat between a curious human and an artifici...,Certainly! A well-chosen color palette is one ...,89b13be2-c82f-493a-9258-1d690440e451,"I am designing a website, can you give me a co...",assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Certainly! A well-chosen color palette is one ...,"I am designing a website, can you give me a co...",69f27058-6253-42bc-892c-5f8a95870ec1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Certainly! A well-chosen color palette is one ...,"USER: I am designing a website, can you give m...",### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
14,A chat between a curious human and an artifici...,Lena was a happy-go-lucky suburban housewife w...,0cd84750-ccf6-4a8e-89bf-02e17a6d0eca,Write me a story synopsis for a novel where th...,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Lena was a happy-go-lucky suburban housewife w...,Write me a story synopsis for a novel where th...,b864688c-62cb-420d-bae1-e81677b4fded,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Lena was a happy-go-lucky suburban housewife w...,USER: Write me a story synopsis for a novel wh...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...
15,A chat between a curious human and an artifici...,Ser introvertido no significa que no puedas te...,b65b2bfa-1fc6-4448-95a5-cc92544d83cf,¿como consigo una novia siendo introvertido?,assistant,True,keyfan/bloomz-rlhf(max_length=512;temperature=...,Ser introvertido no significa que no puedas te...,¿como consigo una novia siendo introvertido?,3855e40b-823f-4b80-811e-e0071034c507,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Ser introvertido no significa que no puedas te...,USER: ¿como consigo una novia siendo introvert...,### TRANSCRIPT OF UNKNOWN FACTUAL ACCURACY ###...


In [40]:
out_str = ""
for row in raw_df.iloc:
    out_str += f"[[MESSAGE_ID]] {row['message_id']}\n"
    out_str += f"[[INPUT_TEXT]] {row['input_text']}\n"
    out_str += "[[NOT YET ANNOTATED]]"
    out_str += "\n\n\n\n"

out_file = f"data/BLANK_val_labels_{model_name}.txt"
with open(out_file, "w") as f:
    f.write(out_str)
out_file

'data/BLANK_val_labels_keyfan_bloomz-rlhf(max_length=512;temperature=1.0;repetition_penalty=1.2;do_sample=True;top_p=0.95).txt'

In [5]:
import re
ipath= u"./data/label-validation/backup/BLANK_val_labels_brennan.txt"
matches = re.findall(r'[\u4e00-\u9fff]+', ipath)
matches

[]